# Curvy-CUSIPS: Swaps Relative Value

In [1]:
import sys
sys.path.append("../")

In [2]:
from CurvyCUSIPs.CurveDataFetcher import CurveDataFetcher
from CurvyCUSIPs.S490Swaps import S490Swaps
from CurvyCUSIPs.USTs import USTs
from CurvyCUSIPs.utils.pca_utils import calc_pca_loadings_matrix

from datetime import datetime
import QuantLib as ql
import pandas as pd
from sklearn.decomposition import PCA
from pandas.tseries.offsets import CustomBusinessDay 
from pandas.tseries.holiday import USFederalHolidayCalendar
from typing import Dict, List

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
plt.style.use('ggplot')
params = {'legend.fontsize': 'x-large',
        'figure.figsize': (18, 10),
        'axes.labelsize': 'x-large',
        'axes.titlesize':'x-large',
        'xtick.labelsize':'x-large',
        'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

import os
from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
config = dotenv_values(env_path)

import nest_asyncio
nest_asyncio.apply()

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

%load_ext autoreload
%autoreload 2

# Set up data fetching infra

In [3]:
curve_data_fetcher = CurveDataFetcher(fred_api_key=config["FRED_API_KEY"])

In [4]:
sofr_ois = S490Swaps(s490_curve_db_path=r"..\db\nyclose_sofr_ois")
usts = USTs(
    cusip_set_db_path=r"..\db\ust_cusip_set",
    cusip_timeseries_db_path=r"..\db\ust_cusip_timeseries",
    ct_eod_db_path=r"..\db\ust_eod_ct_yields",
    curve_data_fetcher=curve_data_fetcher,
)

# Calc Fwd SOFR OIS Grids

In [5]:
start_date = datetime(2018, 1, 1)
end_date = datetime(2024, 12, 24)
bdates = pd.date_range(start=start_date, end=end_date, freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))

In [6]:
fwd_tenors = ["1M", "3M", "6M", "9M", "12M", "18M", "2Y", "3Y", "4Y", "5Y", "6Y", "7Y", "8Y", "9Y", "10Y", "15Y"]
fwd_grid_dict, ql_curves = sofr_ois.s490_nyclose_fwd_curve_matrices(
    start_date=start_date,
    end_date=end_date,
    ql_piecewise_method="logLinearDiscount",
    fwd_tenors=fwd_tenors,
)

Building Implied Fwd Curves...: 100%|██████████| 1749/1749 [00:18<00:00, 92.73it/s] 


In [7]:
dt = datetime(2024, 12, 24)
sofr_ois.fwd_grid_dict_curve_plotter(
    tenor_date_pairs=[("Spot", dt), ("3M Fwd", dt), ("12M Fwd", dt), ("5Y Fwd", dt)],
    fwd_grid_dict=fwd_grid_dict,
    use_plotly=True
)

In [8]:
tenors_to_plot = ["12M Fwd 2Y", "12M Fwd 10Y"]
sofr_ois.fwd_grid_dict_timeseries_plotter(fwd_grid_dict=fwd_grid_dict, tenors_to_plot=tenors_to_plot, bdates=bdates, use_plotly=True)

PLOTTING SWAPS: 100%|██████████| 1749/1749 [00:01<00:00, 1184.06it/s]


# PCA for Asset Selection:
- References:
    - [Standard Chartered Swaps RV Tool](https://github.com/yieldcurvemonkey/Curvy-CUSIPs/blob/main/research/PCA/Introducing_a_relative-value_tool_for_swaps_19_08_13_08_57.pdf)
    - [Credit Suisse PCA Unleashed](https://github.com/yieldcurvemonkey/Curvy-CUSIPs/blob/main/research/PCA/PCA%20Unleashed.%20Interest%20Rate%20Strategy.%20Spot%201y%201y1y%202y1y%203y1y%204y1y%205y1y%206y1y%207y1y%208y1y%209y1y%2010y2y%2012y3y%2015y5y%2020y5y%2025y5y%2030y10y%2040y10y.pdf)
    - [ASM Quant Macro PCA Part II](https://asmquantmacro.com/2015/06/29/principal-component-analysis-part-ii/)
    

In [9]:
pca_results = sofr_ois.pca_on_fwd_grids(
    start_date=datetime(2024, 5, 1),
    end_date=datetime(2024, 12, 24),
    fwd_grid_dict=fwd_grid_dict, 
    rm_swap_tenors=["1D", "1W", "2W", "3W", "1M", "2M", "3M", "4M", "5M", "6M", "7M", "8M", "9M", "10M", "11M"],
    indy_fwd_strips=True,
    rolling_window=20,
    n_jobs=2,
)

# df: pd.DataFrame = pca_results["rich_cheap_zscore_heatmap"]
# df.style.background_gradient(cmap="RdYlGn", axis=0)

Rolling PCA ON INDY FWD STRIPS: 100%|██████████| 17/17 [02:48<00:00,  9.94s/it]


In [14]:
p = pca_results["rolling_pca_results_per_fwd"]["Spot"][datetime(2024, 12, 12)]

In [15]:
p["rich_cheap_zscore_anchor"]

,Residual
Tenor,
12M,-1.493483
18M,1.290040
2Y,1.366077
3Y,1.156549
4Y,1.292248
5Y,2.190115
6Y,1.038026
7Y,-0.672587
8Y,-1.540158


In [ ]:
most_mispriced_dict = sofr_ois.most_mispriced_pca_resid_zscores(df, top_n=10)
display(most_mispriced_dict["curve"])
display(most_mispriced_dict["fly"])

,Forward,Tenor1,Tenor2,ZScore-Spread,Trade
0,4Y Fwd,12M,50Y,-8.393899,flattener
1,4Y Fwd,25Y,50Y,-7.989308,flattener
2,4Y Fwd,12M,6Y,-7.874739,flattener
3,3Y Fwd,18M,7Y,-7.721658,flattener
4,4Y Fwd,12M,7Y,-7.653469,flattener
5,3Y Fwd,18M,8Y,-7.642601,flattener
6,3Y Fwd,2Y,7Y,-7.503460,flattener
7,3Y Fwd,18M,50Y,-7.502743,flattener
8,4Y Fwd,6Y,25Y,7.470148,steepener
9,4Y Fwd,20Y,50Y,-7.426773,flattener


,Forward,ShortWing,Belly,LongWing,ZScore-Spread,Trade
0,4Y Fwd,6Y,25Y,50Y,-15.459456,pay belly
1,4Y Fwd,12M,6Y,25Y,15.344887,pay belly
2,4Y Fwd,7Y,25Y,50Y,-15.238186,pay belly
3,4Y Fwd,12M,7Y,25Y,14.902348,pay belly
4,4Y Fwd,5Y,25Y,50Y,-14.889232,pay belly
5,4Y Fwd,12M,6Y,20Y,14.782352,pay belly
6,4Y Fwd,12M,6Y,30Y,14.696318,pay belly
7,3Y Fwd,18M,7Y,25Y,14.419378,pay belly
8,4Y Fwd,12M,7Y,20Y,14.339813,pay belly
9,4Y Fwd,6Y,20Y,50Y,-14.334387,pay belly


In [354]:
sofr_ois.pca_residual_timeseries_plotter(pca_results=pca_results, tenors_to_plot=["12M Fwd 4Y", "12M Fwd 9Y"], use_plotly=True)

PLOTTING PCA RESIDS: 100%|██████████| 246/246 [00:00<00:00, 893.85it/s]


In [407]:
sofr_ois.pca_residual_credit_suisse_BBar_plot(
    pca_results=pca_results,
    tenors_to_plot=[
        "2Y",
        "3Y",
        "4Y",
        "5Y",
        "6Y",
        "7Y",
        "8Y",
        "9Y",
        "10Y",
        "12Y",
        "15Y",
        "20Y",
        "25Y",
        "30Y",
        "40Y",
        "50Y",
        "12M Fwd 12M",
        "12M Fwd 2Y",
        "12M Fwd 3Y",
        "12M Fwd 4Y",
        "12M Fwd 5Y",
        "12M Fwd 6Y",
        "12M Fwd 7Y",
        "12M Fwd 8Y",
        "12M Fwd 9Y",
        "12M Fwd 10Y",
        "12M Fwd 12Y",
        "12M Fwd 15Y",
        "12M Fwd 20Y",
        "12M Fwd 25Y",
        "12M Fwd 30Y",
        "12M Fwd 40Y",
        "12M Fwd 50Y",
    ],
    bday_offsets=[20, 5], 
    title="2024-12-24 PCA Residuals from YTD Spot & 12M Fwd SOFR OIS Daily Rates with 1W & 1M Old Residuals Overlaid (skinner & lighter => older)"
)